In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
from torch.optim import AdamW
from torch.utils.data import DataLoader

from transformers import AutoTokenizer
from datasets import load_dataset

from tklearn.metrics import Accuracy
from tklearn.nn import Trainer, Evaluator
from tklearn.nn.callbacks import ProgbarLogger, EarlyStopping
from tklearn.nn.prototypes import PrototypeForSequenceClassification, PrototypeCallback

In [3]:
MODEL_NAME_OR_PATH = "google-bert/bert-base-uncased"
DATASET = "yelp_review_full"

In [4]:
dataset = load_dataset(DATASET)

dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [6]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [7]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

In [8]:
model = PrototypeForSequenceClassification.from_pretrained(MODEL_NAME_OR_PATH)

model.to("mps")

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=16)

In [9]:
valid_dataloader = DataLoader(small_eval_dataset, batch_size=32)

evaluator = Evaluator(model, valid_dataloader, callbacks=[PrototypeCallback(train_dataloader), ProgbarLogger()], metrics={"acuracy": Accuracy()}, prefix="valid_")

In [10]:
optimizer = AdamW(model.parameters(), lr=1e-5)

evaluator = Evaluator(model, valid_dataloader, callbacks=[PrototypeCallback(train_dataloader), ProgbarLogger()], metrics={"acuracy": Accuracy()}, prefix="valid_")

trainer = Trainer(model, train_dataloader, optimizer=optimizer, callbacks=[ProgbarLogger(), EarlyStopping(patience=0)], evaluator=evaluator, epochs=10)

In [11]:
trainer.train()

VBox()

VBox()

VBox()

VBox()

VBox()